In [1]:
# dependancies
import pandas as pd
import numpy as np
import re
import psycopg2
import time

In [30]:
file_dir = '/Users/xenia/Data Bootcamp/Analysis_Projects/capstone/Raw_Data'

athlets_data = pd.read_csv(f'{file_dir}/olympic_athletes.csv', low_memory=False)
athlets_data

,athlete_url,athlete_full_name,first_game,athlete_year_birth,athlete_medals,games_participations,bio
0,https://olympics.com/en/athletes/dongqi-chen,Dongqi CHEN,Tokyo 2020,1988.0,NaN,NaN,NaN
1,https://olympics.com/en/athletes/sagen-maddalena,Sagen MADDALENA,Tokyo 2020,1993.0,NaN,NaN,NaN
2,https://olympics.com/en/athletes/maria-martynova,Maria MARTYNOVA,Tokyo 2020,1997.0,NaN,NaN,NaN
3,https://olympics.com/en/athletes/yulia-karimova,Yulia KARIMOVA,Tokyo 2020,1994.0,\n\n\n2\n\nB\n\n,1.0,NaN
4,https://olympics.com/en/athletes/najmeh-khedmati,Najmeh KHEDMATI,Rio 2016,1996.0,NaN,3.0,NaN
...,...,...,...,...,...,...,...
74726,https://olympics.com/en/athletes/douglas-weigle,Douglas WEIGLE,Innsbruck 1976,1955.0,NaN,1.0,\n\n\nKent Weigle competed in ice dancing with...
74727,https://olympics.com/en/athletes/stefania-bertele,Stefania BERTELE,Innsbruck 1976,1957.0,NaN,1.0,\n\n\nStefania Bertele had the following place...
74728,https://olympics.com/en/athletes/walter-cecconi,Walter CECCONI,Innsbruck 1976,1957.0,NaN,1.0,\n\n\nWalter Cecconi had the following placeme...
74729,https://olympics.com/en/athletes/susan-kelley,Susan KELLEY,Innsbruck 1976,1954.0,NaN,1.0,\n\n\nSusie Kelley competed in ice dancing wit...


In [31]:
athlets_data.dtypes

athlete_url              object
athlete_full_name        object
first_game               object
athlete_year_birth      float64
athlete_medals           object
games_participations    float64
bio                      object
dtype: object

In [32]:
athlets_data = athlets_data.drop(["bio", "athlete_url"], axis = 1)
athlets_data

,athlete_full_name,first_game,athlete_year_birth,athlete_medals,games_participations
0,Dongqi CHEN,Tokyo 2020,1988.0,NaN,NaN
1,Sagen MADDALENA,Tokyo 2020,1993.0,NaN,NaN
2,Maria MARTYNOVA,Tokyo 2020,1997.0,NaN,NaN
3,Yulia KARIMOVA,Tokyo 2020,1994.0,\n\n\n2\n\nB\n\n,1.0
4,Najmeh KHEDMATI,Rio 2016,1996.0,NaN,3.0
...,...,...,...,...,...
74726,Douglas WEIGLE,Innsbruck 1976,1955.0,NaN,1.0
74727,Stefania BERTELE,Innsbruck 1976,1957.0,NaN,1.0
74728,Walter CECCONI,Innsbruck 1976,1957.0,NaN,1.0
74729,Susan KELLEY,Innsbruck 1976,1954.0,NaN,1.0


In [77]:
athlets_data['games_participations'] = athlets_data['games_participations'].fillna(0) # assuming NA means the athlete neve partisipated in games before
athlets_data['athlete_year_birth'] = athlets_data['athlete_year_birth'].fillna(0) 
athlets_data['athlete_medals'] = athlets_data['athlete_medals'].fillna(0)
# in order to conver year to integer and check how many athlets does not have the year of birth
# convert float64 to int for 'athlete_year_birth' & 'games_participations'
athlets_data['games_participations'] = athlets_data['games_participations'].astype(int)
athlets_data['athlete_year_birth'] = athlets_data['athlete_year_birth'].astype(int)
athlets_data

,athlete_full_name,first_game,athlete_year_birth,athlete_medals,games_participations
0,Dongqi CHEN,Tokyo 2020,1988,0,0
1,Sagen MADDALENA,Tokyo 2020,1993,0,0
2,Maria MARTYNOVA,Tokyo 2020,1997,0,0
3,Yulia KARIMOVA,Tokyo 2020,1994,\n\n\n2\n\nB\n\n,1
4,Najmeh KHEDMATI,Rio 2016,1996,0,3
...,...,...,...,...,...
74726,Douglas WEIGLE,Innsbruck 1976,1955,0,1
74727,Stefania BERTELE,Innsbruck 1976,1957,0,1
74728,Walter CECCONI,Innsbruck 1976,1957,0,1
74729,Susan KELLEY,Innsbruck 1976,1954,0,1


In [34]:
athlets_data[athlets_data['athlete_year_birth'] == 0].shape 

(2444, 5)

In [82]:
# since there are about 3% of athllets dosent have BD in their profile it is looks reasonable to drop them 
athlets_data_drop_bd = athlets_data[athlets_data['athlete_year_birth'] != 0]
athlets_data_drop_bd

,athlete_full_name,first_game,athlete_year_birth,athlete_medals,games_participations
0,Dongqi CHEN,Tokyo 2020,1988,0,0
1,Sagen MADDALENA,Tokyo 2020,1993,0,0
2,Maria MARTYNOVA,Tokyo 2020,1997,0,0
3,Yulia KARIMOVA,Tokyo 2020,1994,\n\n\n2\n\nB\n\n,1
4,Najmeh KHEDMATI,Rio 2016,1996,0,3
...,...,...,...,...,...
74726,Douglas WEIGLE,Innsbruck 1976,1955,0,1
74727,Stefania BERTELE,Innsbruck 1976,1957,0,1
74728,Walter CECCONI,Innsbruck 1976,1957,0,1
74729,Susan KELLEY,Innsbruck 1976,1954,0,1


In [83]:
athlets_data_drop_bd['athlete_medals'].value_counts()

0                                               59152
\n\n\n1\n\nB\n\n                                 3568
\n\n\n1\n\nS\n\n                                 2904
\n\n\n1\n\nG\n\n                                 2288
\n\n\n1\n\nG\n\n\n\n1\n\nS\n\n                    562
                                                ...  
\n\n\n3\n\nS\n\n\n\n6\n\nB\n\n                      1
\n\n\n5\n\nG\n\n\n\n3\n\nS\n\n                      1
\n\n\n5\n\nG\n\n\n\n2\n\nS\n\n\n\n4\n\nB\n\n        1
\n\n\n4\n\nG\n\n\n\n3\n\nS\n\n                      1
\n\n\n9\n\nG\n\n\n\n1\n\nS\n\n\n\n1\n\nB\n\n        1
Name: athlete_medals, Length: 170, dtype: int64

In [84]:
medals_data = pd.read_csv(f'{file_dir}/olympic_medals.csv', low_memory=False)
medals_data #doesn't seem usefull


,discipline_title,slug_game,event_title,event_gender,medal_type,participant_type,participant_title,athlete_url,athlete_full_name,country_name,country_code,country_3_letter_code
0,Shooting,tokyo-2020,Trap Mixed Team,Mixed,GOLD,GameTeam,Spain,https://olympics.com/en/athletes/fatima-galvez,Fatima GALVEZ,Spain,ES,ESP
1,Shooting,tokyo-2020,Trap Mixed Team,Mixed,GOLD,GameTeam,Spain,https://olympics.com/en/athletes/alberto-ferna...,Alberto FERNANDEZ,Spain,ES,ESP
2,Shooting,tokyo-2020,Trap Mixed Team,Mixed,SILVER,GameTeam,San Marino,https://olympics.com/en/athletes/alessandra-pe...,Alessandra PERILLI,San Marino,SM,SMR
3,Shooting,tokyo-2020,Trap Mixed Team,Mixed,SILVER,GameTeam,San Marino,https://olympics.com/en/athletes/gian-marco-berti,Gian Marco BERTI,San Marino,SM,SMR
4,Shooting,tokyo-2020,Trap Mixed Team,Mixed,BRONZE,GameTeam,United States of America,https://olympics.com/en/athletes/madelynn-ann-...,Madelynn Ann BERNAU,United States of America,US,USA
...,...,...,...,...,...,...,...,...,...,...,...,...
21305,Weightlifting,athens-1896,heavyweight - one hand lift men,Men,SILVER,Athlete,NaN,https://olympics.com/en/athletes/viggo-jensen,Viggo JENSEN,Denmark,DK,DEN
21306,Weightlifting,athens-1896,heavyweight - one hand lift men,Men,BRONZE,Athlete,NaN,NaN,Alexandros Nikolopoulos,Greece,GR,GRE
21307,Weightlifting,athens-1896,heavyweight - two hand lift men,Men,GOLD,Athlete,NaN,https://olympics.com/en/athletes/viggo-jensen,Viggo JENSEN,Denmark,DK,DEN
21308,Weightlifting,athens-1896,heavyweight - two hand lift men,Men,SILVER,Athlete,NaN,https://olympics.com/en/athletes/launceston-el...,Launceston ELLIOT,Great Britain,GB,GBR


In [87]:
athlets_data_drop_bd['athlete_medals'] = athlets_data_drop_bd['athlete_medals'].str.lstrip().str.rstrip().replace('\s+', '', regex=True)

athlets_data_drop_bd

/Users/xenia/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,athlete_full_name,first_game,athlete_year_birth,athlete_medals,games_participations
0,Dongqi CHEN,Tokyo 2020,1988,NaN,0
1,Sagen MADDALENA,Tokyo 2020,1993,NaN,0
2,Maria MARTYNOVA,Tokyo 2020,1997,NaN,0
3,Yulia KARIMOVA,Tokyo 2020,1994,2B,1
4,Najmeh KHEDMATI,Rio 2016,1996,NaN,3
...,...,...,...,...,...
74726,Douglas WEIGLE,Innsbruck 1976,1955,NaN,1
74727,Stefania BERTELE,Innsbruck 1976,1957,NaN,1
74728,Walter CECCONI,Innsbruck 1976,1957,NaN,1
74729,Susan KELLEY,Innsbruck 1976,1954,NaN,1


In [88]:
athlets_data_drop_bd['athlete_medals'].value_counts()

1B        3568
1S        2904
1G        2288
1G1S       562
1S1B       549
          ... 
8G4S1B       1
7G4S         1
9G1S         1
5G2S1B       1
5G1S3B       1
Name: athlete_medals, Length: 169, dtype: int64

In [106]:
athlets_data['athlete_medals'] = athlets_data['athlete_medals'].astype(str)
medals = athlets_data_drop_bd['athlete_medals'].tolist()
medals

[nan,
 nan,
 nan,
 '2B',
 nan,
 '1G1B',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 '1S',
 nan,
 nan,
 nan,
 '1B',
 nan,
 nan,
 nan,
 nan,
 nan,
 '1B',
 '1S',
 '2S',
 '1S',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 '1B',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 '1G1S',
 '1G1S',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 '1G1B',
 nan,
 '2B',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 '1S',
 nan,
 '1G',
 '1G1B',
 nan,
 nan,
 nan,
 '1B',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 '1S',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 '2G',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 '1S',
 nan,
 nan,
 nan,
 '1G',
 '1S',
 '1G',
 nan,
 nan,
 '1G',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,


In [121]:
gold_medals = []
silver_medals = []
bronze_medals = []

gold_pattern = r'(\d+)G'
silver_pattern = r'(\d+)S'
bronze_pattern = r'(\d+)B'
gold = 0

for medal in medals:
    if pd.isna(medal):
        gold_medals.append('0')
        silver_medals.append('0')
        bronze_medals.append('0')
        continue
        
    if re.search(gold_pattern, medal):
        m=re.search(gold_pattern, medal)
        print(m.group(1))
        print(m.group(0))
        continue
    else: 
        gold_medals.append(0)
    if re.search(silver_pattern, medal):
        m=re.search(silver_pattern, medal)
        print(m)
        continue
    else:
        silver_medals.append(0)
    if re.search(bronze_pattern, medal):
        m=re.search(bronze_pattern, medal)
        print(m)
        continue
    else:
        bronze_medals.append(0)

print(gold_medals)

<re.Match object; span=(0, 2), match='2B'>
1
1G
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1B'>
<re.Match object; span=(0, 2), match='1B'>
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='2S'>
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1B'>
1
1G
1
1G
1
1G
<re.Match object; span=(0, 2), match='2B'>
<re.Match object; span=(0, 2), match='1S'>
1
1G
1
1G
<re.Match object; span=(0, 2), match='1B'>
<re.Match object; span=(0, 2), match='1S'>
2
2G
<re.Match object; span=(0, 2), match='1S'>
1
1G
<re.Match object; span=(0, 2), match='1S'>
1
1G
1
1G
<re.Match object; span=(0, 2), match='1B'>
<re.Match object; span=(0, 2), match='1B'>
1
1G
1
1G
1
1G
<re.Match object; span=(0, 2), match='1B'>
1
1G
1
1G
<re.Match object; span=(0, 2), match='1B'>
1
1G
1
1G
1
1G
<re.Match object; span=(0, 2), match='1B'>
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1S'>
<re.Mat

1G
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='4S'>
1
1G
<re.Match object; span=(0, 2), match='1B'>
<re.Match object; span=(0, 2), match='1B'>
1
1G
<re.Match object; span=(0, 2), match='1B'>
<re.Match object; span=(0, 2), match='1S'>
1
1G
<re.Match object; span=(0, 2), match='1B'>
2
2G
<re.Match object; span=(0, 2), match='1S'>
1
1G
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1S'>
1
1G
2
2G
<re.Match object; span=(0, 2), match='1B'>
<re.Match object; span=(0, 2), match='2S'>
<re.Match object; span=(0, 2), match='1B'>
<re.Match object; span=(0, 2), match='1S'>
1
1G
<re.Match object; span=(0, 2), match='1B'>
1
1G
<re.Match object; span=(0, 2), match='1S'>
1
1G
<re.Match object; span=(0, 2), match='1B'>
<re.Match object; span=(0, 2), match='1B'>
1
1G
<re.Match object; span=(0, 2), match='1S'>
1
1G
<re.Match object; span=(0, 2), match='1B'>
<re.Match object; span=(0, 2), match='1B'>
1
1G
<re.Match object; span=(0, 2)

2
2G
1
1G
1
1G
1
1G
<re.Match object; span=(0, 2), match='1S'>
23
23G
6
6G
<re.Match object; span=(0, 2), match='1S'>
1
1G
2
2G
1
1G
1
1G
1
1G
1
1G
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1B'>
<re.Match object; span=(0, 2), match='1B'>
5
5G
<re.Match object; span=(0, 2), match='1B'>
<re.Match object; span=(0, 2), match='1B'>
2
2G
<re.Match object; span=(0, 2), match='1B'>
1
1G
3
3G
5
5G
<re.Match object; span=(0, 2), match='1B'>
<re.Match object; span=(0, 2), match='3B'>
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1S'>
1
1G
1
1G
<re.Match object; span=(0, 2), match='1S'>
2
2G
2
2G
2
2G
<re.Match object; span=(0, 2), match='1S'>
1
1G
<re.Match object; span=(0, 2), match='1S'>
1
1G
2
2G
1
1G
1
1G
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1B'>
1
1G
<re.Match object; span=(0, 2), match='1S'>
4
4G
4
4G
2
2G
<re.Match object; span=(0, 2), match='1S'>
2
2G
<re.Match object; spa

1G
<re.Match object; span=(0, 2), match='1B'>
<re.Match object; span=(0, 2), match='1S'>
1
1G
<re.Match object; span=(0, 2), match='1S'>
1
1G
<re.Match object; span=(0, 2), match='1S'>
1
1G
1
1G
2
2G
<re.Match object; span=(0, 2), match='3S'>
1
1G
<re.Match object; span=(0, 2), match='1B'>
1
1G
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='2B'>
3
3G
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1B'>
<re.Match object; span=(0, 2), match='1B'>
1
1G
<re.Match object; span=(0, 2), match='1B'>
2
2G
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1B'>
<re.Match object; span=(0, 2), match='2S'>
<re.Match object; span=(0, 2), match='1B'>
<re.Match object; span=(0, 2), match='2S'>
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1B'>
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1S'>
1
1G
1
1G
<re.Match object; span=(0, 2), mat

1
1G
<re.Match object; span=(0, 2), match='1B'>
<re.Match object; span=(0, 2), match='1B'>
1
1G
<re.Match object; span=(0, 2), match='1B'>
1
1G
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1B'>
1
1G
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1B'>
1
1G
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1B'>
<re.Match object; span=(0, 2), match='1B'>
1
1G
1
1G
1
1G
2
2G
<re.Match object; span=(0, 2), match='1B'>
1
1G
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1B'>
<re.Match object; span=(0, 2), match='1S'>
1
1G
<re.Match object; span=(0, 2), match='1B'>
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1S'>
1
1G
<re.Match object; span=(0, 2), match='1B'>
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match=

...

<re.Match object; span=(0, 2), match='2S'>
<re.Match object; span=(0, 2), match='2B'>
1
1G
<re.Match object; span=(0, 2), match='2B'>
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1S'>
2
2G
<re.Match object; span=(0, 2), match='1S'>
1
1G
<re.Match object; span=(0, 2), match='1S'>
1
1G
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1B'>
2
2G
<re.Match object; span=(0, 2), match='1S'>
1
1G
1
1G
<re.Match object; span=(0, 2), match='1S'>
1
1G
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1B'>
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1B'>
<re.Match object; span=(0, 2), match='1S'>
1
1G
<re.Match object; span=(0, 2), match='1B'>
1
1G
<re.Match object; span=(0, 2), match='1B'>
1
1G
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1B'>
2
2G
6
6G
3
3G
1
1G
<re.Match object; span=(0, 2), match='2B'>
1
1G
2
2G
2
2G
<re.Mat

1
1G
<re.Match object; span=(0, 2), match='1S'>
1
1G
<re.Match object; span=(0, 2), match='1S'>
1
1G
1
1G
<re.Match object; span=(0, 2), match='1B'>
1
1G
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1B'>
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1B'>
1
1G
<re.Match object; span=(0, 2), match='1S'>
1
1G
1
1G
1
1G
<re.Match object; span=(0, 2), match='1S'>
2
2G
<re.Match object; span=(0, 2), match='2S'>
1
1G
<re.Match object; span=(0, 2), match='1S'>
1
1G
2
2G
<re.Match object; span=(0, 2), match='1S'>
3
3G
5
5G
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1S'>
1
1G
4
4G
2
2G
<re.Match object; span=(0, 2), match='1S'>
2
2G
1
1G
<re.Match object; span=(0, 2), match='1S'>
4
4G
2
2G
2
2G
2
2G
<re.Match object; span=(0, 2), match='2B'>
<re.Match object; span=(0, 2), match='1B'>
2
2G
<re.Match object; span=(0, 2), match='2S'>
<re.Match objec

<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1B'>
5
5G
4
4G
<re.Match object; span=(0, 2), match='1S'>
1
1G
1
1G
<re.Match object; span=(0, 2), match='1B'>
<re.Match object; span=(0, 2), match='1S'>
1
1G
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1B'>
1
1G
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1B'>
<re.Match object; span=(0, 2), match='1B'>
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1S'>
1
1G
1
1G
1
1G
1
1G
1
1G
1
1G
1
1G
1
1G
1
1G
1
1G
1
1G
<re.Match object; span=(0, 2), match='1B'>
1
1G
3
3G
<re.Match object; span=(0, 2), match='1B'>
1
1G
<re.Match object; span=(0, 2), match='1B'>
1
1G
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), matc

<re.Match object; span=(0, 2), match='1B'>
<re.Match object; span=(0, 2), match='1S'>
1
1G
1
1G
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='2B'>
<re.Match object; span=(0, 2), match='1B'>
<re.Match object; span=(0, 2), match='1S'>
3
3G
1
1G
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1B'>
<re.Match object; span=(0, 2), match='1B'>
<re.Match object; span=(0, 2), match='2S'>
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1B'>
<re.Match object; span=(0, 2), match='1B'>
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='3S'>
<re.Match object; span=(0, 2), match='1S'>
2
2G
<re.Match object; span=(0, 2), match='1B'>
<re.Match object; span=(0, 2), match='1B'>
<re.Match object; span=(0, 2), match='2S'>
1
1G
2
2G
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1B'>
2
2G
<re.Match obje

<re.Match object; span=(0, 2), match='1B'>
<re.Match object; span=(0, 2), match='2B'>
1
1G
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1S'>
1
1G
<re.Match object; span=(0, 2), match='1B'>
<re.Match object; span=(0, 2), match='1B'>
<re.Match object; span=(0, 2), match='1B'>
1
1G
1
1G
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1B'>
<re.Match object; span=(0, 2), match='1B'>
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1S'>
1
1G
2
2G
<re.Match object; span=(0, 2), match='1B'>
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1B'>
1
1G
1
1G
<re.Match object; span=(0, 2), match='2S'>
<re.Match object; span=(0, 2), match='1S'>
1
1G
1
1G
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1S'>
<re.Match object; span=(0, 2), match='1S'>
<re.